# Modern Portfolio Theory
### UTD Quantitative Finance Club
Finding the Best Portfolio...

In [ ]:
# Library installation (Only run if necessary)
!pip install yfinance matplotlib pandas numpy seaborn tqdm

print("\nInstalled all requirements\n")

In [ ]:
# Imports
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm

print("Imported Successfully!")

# Finding the Mathematical 'Best'
Lets start by analyzing a singular stock to define what 'Best' means.

In [ ]:
ticker = input()

stock = yf.Ticker(ticker)
data = stock.history(period="1y")

In [ ]:
plt.figure(figsize=(19, 6))
plt.plot(data.index, data['Close'], label=f'{ticker} Closing Price')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.title(f'{ticker} Closing Price')
plt.grid()
plt.show()

## Questions
* What constitutes Risk and how do we measure it?
* What constitutes Reward and how do we measure it?

In [ ]:
returns = data['Close'].pct_change().dropna()
mean_return = returns.mean()
volatility = returns.std()

plt.figure(figsize=(18, 6))
plt.scatter(data.index[1:], returns, label=f'{ticker} Daily Returns')
plt.plot(data.index[1:], returns, alpha=0.2)
plt.axhline(mean_return, color='g', linestyle='--', label=f'Mean Return: {mean_return:.4f}')
plt.axhline(mean_return + volatility, color='r', linestyle='--', label=f'Volatility Upper Bound: {mean_return + volatility:.4f}')
plt.axhline(mean_return - volatility, color='r', linestyle='--', label=f'Volatility Lower Bound: {mean_return - volatility:.4f}')

plt.xlabel('Date')
plt.ylabel('Returns')
plt.title(f'{ticker} Returns - Last 1 Year')
plt.legend()
plt.grid()
plt.show()

# Mathematical Definition of Sharpe Ratio

In [ ]:
risk_free_rate = 0.004
sharpe_ratio = (mean_return * 252 - risk_free_rate) / (volatility * np.sqrt(252))
print("Sharpe Ratio: " + str(sharpe_ratio))

# Lets find Sharpe Ratios for various Stocks
* Are they Different? How different?
* Can sharpe Ratio be Negative? What dies that imply?

In [ ]:
def calculate_sharpe_ratio_from_ticker(ticker, risk_free_rate=0.04):
    stock_data = yf.download(ticker, start='2020-01-01', end='2021-01-01')
    daily_returns = stock_data['Close'].pct_change().dropna()
    mean_return = daily_returns.mean()
    std_dev = daily_returns.std()

    annualized_return = mean_return * 252
    annualized_volatility = std_dev * np.sqrt(252)

    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility
    return sharpe_ratio

while True:
    ticker = input("Enter stock ticker (or type 'exit' to quit): ").strip()
    if ticker.lower() == "exit":
        break

    sharpe_ratio = calculate_sharpe_ratio_from_ticker(ticker)
    print("\nSHARPE RATIO FOR " + str(ticker) + ": " + str(sharpe_ratio.iloc[0]) + "\n")


# Measuring Sharpe Ratio for a Portfolio

In [ ]:
assets = ['AMZN', 'JNJ', 'O', 'CHGG', 'JEPI']
print("Assets: " + str(assets))

In [ ]:
start_date = '2023-01-01'
end_date = '2024-01-01'

data = pd.DataFrame()

for asset in assets:
  asset_data = yf.download(asset, start=start_date, end=end_date)['Close']
  data[asset] = asset_data

fig, axs = plt.subplots(1, len(assets), figsize=(20, 6))

for i, asset in enumerate(assets):
  axs[i].plot(data.index, data[asset], label=asset)
  axs[i].set_title(f"{asset} Close Prices")
  axs[i].set_xlabel("Date")
  axs[i].set_ylabel("Close Price")
  axs[i].legend()
  axs[i].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
returns = data.pct_change().dropna()
mean_returns = returns.mean()
cov_matrix = returns.cov()
diag_sqrt = np.sqrt(np.diag(cov_matrix))
normalized_cov_matrix = cov_matrix / np.outer(diag_sqrt, diag_sqrt)

fig, axs = plt.subplots(1, 2, figsize=(19, 6))
sns.barplot(x=mean_returns.index, y=mean_returns.values, ax=axs[0])
axs[0].set_title('Mean Returns')
axs[0].set_xlabel('Asset')
axs[0].set_ylabel('Mean Return')
axs[0].tick_params(axis='x', rotation=45)
axs[0].grid(True)
sns.heatmap(normalized_cov_matrix, annot=True, cmap='coolwarm', fmt="4f",vmin=-1, vmax=1, ax=axs[1])
axs[1].set_title('Normalized Covariance Matrix - Returns')
axs[1].set_xlabel('Asset')
axs[1].set_ylabel('Asset')

# Numerical Solution to the Weight Optimization Problem

In [ ]:
risk_free_rate_annualized = risk_free_rate
annualized_cov_matrix = cov_matrix * 252

num_portfolios = 50000
results = np.zeros((3 + len(assets), num_portfolios))

for i in tqdm(range(num_portfolios)):

    weights = np.random.uniform(0, 1, size=len(assets))
    weights /= np.sum(weights)

    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(annualized_cov_matrix, weights)))

    results[0, i] = portfolio_return
    results[1, i] = portfolio_std_dev
    results[2, i] = (portfolio_return - risk_free_rate_annualized) / portfolio_std_dev

    for j in range(len(weights)):
        results[j+3, i] = weights[j]

columns = ['Return', 'Volatility', 'Sharpe'] + list(assets)
results_df = pd.DataFrame(results.T, columns=columns)

ones = np.ones(len(assets))
inv_cov = np.linalg.inv(annualized_cov_matrix)
excess_returns = (mean_returns * 252) - risk_free_rate_annualized

optimal_weights = np.dot(inv_cov, excess_returns) / np.dot(ones.T, np.dot(inv_cov, excess_returns))

optimal_return = np.sum(mean_returns * optimal_weights) * 252
optimal_volatility = np.sqrt(np.dot(optimal_weights.T, np.dot(annualized_cov_matrix, optimal_weights)))
optimal_sharpe = (optimal_return - risk_free_rate_annualized) / optimal_volatility

best_sharpe_idx = results_df['Sharpe'].idxmax()
best_sharpe = results_df.iloc[best_sharpe_idx]['Sharpe']
best_return = results_df.iloc[best_sharpe_idx]['Return']
best_volatility = results_df.iloc[best_sharpe_idx]['Volatility']

plt.figure(figsize=(21, 8))
scatter = plt.scatter(results_df['Volatility'], results_df['Return'], c=results_df['Sharpe'], cmap='inferno', alpha=0.6, s=2)
plt.colorbar(scatter, label='Sharpe Ratio')

plt.scatter(best_volatility, best_return, color='black', marker='*', s=250, label="Best Monte Carlo Portfolio", edgecolors='black')

plt.xlabel('Standard Deviation (Volatility)')
plt.ylabel('Annualized Return')
plt.title('Portfolio Return vs. Portfolio Volatility')
plt.legend()
plt.grid(True)
plt.show()

## Questions
* Why is the portfolio with the optimal sharpe ratio where it is?
* What if I wanted a portfolio with maximim returns?
* what if I wanted the most risk free portfolio?
* Which portfolios are sub-optimal and should not be used? Why?

In [ ]:
min_vol_idx = results_df['Volatility'].idxmin()
min_vol_weights = results_df.iloc[min_vol_idx][3:].values
min_vol_sharpe = results_df.iloc[min_vol_idx]['Sharpe']

max_return_idx = results_df['Return'].idxmax()
max_return_weights = results_df.iloc[max_return_idx][3:].values
max_return_sharpe = results_df.iloc[max_return_idx]['Sharpe']

max_sharpe_weights = results_df.iloc[best_sharpe_idx][3:].values
max_sharpe_ratio = results_df.iloc[best_sharpe_idx]['Sharpe']

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].pie(max_sharpe_weights, labels=assets, autopct='%1.1f%%', startangle=140)
axes[0].set_title(f"Max Sharpe Ratio Portfolio (Sharpe: {max_sharpe_ratio:.5f})")

axes[1].pie(min_vol_weights, labels=assets, autopct='%1.1f%%', startangle=140)
axes[1].set_title(f"Min Variance Portfolio (Sharpe: {min_vol_sharpe:.5f})")

axes[2].pie(max_return_weights, labels=assets, autopct='%1.1f%%', startangle=140)
axes[2].set_title(f"Max Return Portfolio (Sharpe: {max_return_sharpe:.5f})")

plt.tight_layout()
plt.show()

# GBM (Geometric Brownian Motion) Basics: Singular Stock

In [ ]:
ticker = "MAIN"

mu = 0.01
sigma = 0.01
M = 1

stock_data = yf.download(ticker, start="2022-01-01", end="2023-06-01", interval="1d")
stock_data["Days"] = np.arange(len(stock_data))


T = 147 / 252
dt = 1/252
n_steps = int(T / dt)
np.random.seed()



S0 = stock_data["Close"].iloc[-1]

time = np.linspace(0, T, n_steps)

simulations = np.zeros((M, n_steps))

for m in range(M):
    S = np.zeros(n_steps)
    S[0] = S0
    for t in range(1, n_steps):
        dW = np.random.normal(0, np.sqrt(dt))
        S[t] = S[t-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * dW)
    simulations[m, :] = S

plt.figure(figsize=(18, 8))


plt.plot(stock_data['Days'], stock_data['Close'], label="Historical Data", color='black')

for m in range(M):
    plt.plot(time * 252 + stock_data['Days'].iloc[-1], simulations[m, :], color='blue', alpha=0.4)

plt.title(f"{ticker} Historical Stock Prices & {M} GBM Simulations (Next 200 Days)")
plt.xlabel("Days")
plt.ylabel("Stock Price")
plt.grid(True)
plt.legend()
plt.show()

# Forecasting Different Portfolios
* Portfolio Adapted Geometric Brownian Motion

In [ ]:
T = int(252)
n_simulations = 5000
dt = 1

S0 = 1000


def simulate_risk_free_rate(start):
    rfr = np.zeros(T)
    for i in range(0, T):
        rfr[i] = start * np.exp((0.04/T) * i)

    return rfr

def simulate_gbm(S0, mu, sigma, T, n_simulations):
    np.random.seed()
    paths = np.zeros((T, n_simulations))
    paths[0] = S0
    for t in range(1, T):
        Z = np.random.normal(0, 1, n_simulations)
        paths[t] = paths[t-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
    return paths

rfr = simulate_risk_free_rate(S0)

gbm_max_sharpe = simulate_gbm(S0, best_return/252, best_volatility/np.sqrt(252), T, n_simulations)
gbm_min_vol = simulate_gbm(S0, results_df.iloc[min_vol_idx]['Return']/252, results_df.iloc[min_vol_idx]['Volatility']/np.sqrt(252), T, n_simulations)
gbm_max_return = simulate_gbm(S0, results_df.iloc[max_return_idx]['Return']/252, results_df.iloc[max_return_idx]['Volatility']/np.sqrt(252), T, n_simulations)

fig, axes = plt.subplots(3, 1, figsize=(18, 10), sharex=True)

for i in tqdm(range(n_simulations)):
    axes[0].plot(gbm_max_sharpe[:, i], alpha=0.05, color='blue')
    axes[0].plot(rfr, alpha=0.4, color='black')

    axes[1].plot(gbm_min_vol[:, i], alpha=0.05, color='green')
    axes[1].plot(rfr, alpha=0.4, color='black')

    axes[2].plot(gbm_max_return[:, i], alpha=0.05, color='red')
    axes[2].plot(rfr, alpha=0.4, color='black')

axes[0].set_title("GBM Simulation - Max Sharpe Ratio Portfolio")
axes[1].set_title("GBM Simulation - Min Variance Portfolio")
axes[2].set_title("GBM Simulation - Max Return Portfolio")

for ax in axes:
    ax.set_ylabel("Portfolio Value")
    ax.grid()

plt.xlabel("Time Steps (Days)")
plt.tight_layout()
plt.show()


In [ ]:
final_values_max_sharpe = gbm_max_sharpe[-1, :]
final_values_min_vol = gbm_min_vol[-1, :]
final_values_max_return = gbm_max_return[-1, :]

plt.figure(figsize=(17, 7))
plt.boxplot([final_values_max_sharpe, final_values_min_vol, final_values_max_return],
            labels=["Max Sharpe", "Min Volatility", "Max Return"],
            patch_artist=True,
            boxprops=dict(facecolor='white', color='blue'),
            whiskerprops=dict(color='blue'),
            medianprops=dict(color='red'))

plt.axhline(y=rfr[-1], color='green', linestyle='--', linewidth=2)

plt.title("Box and Whisker Plot of Portfolio Final Values")
plt.xlabel("Portfolio Type")
plt.ylabel("Final Portfolio Value (End of Year)")

plt.grid(True)
plt.tight_layout()
plt.show()


# Short Selling
* Can we model short selling as a negative weight?
* can we construct a portfolio that is more optimal with negative weights? (short selling)
* How does this look?

In [ ]:
annualized_cov_matrix = cov_matrix * 252

num_portfolios = 1000000
results = np.zeros((3 + len(assets), num_portfolios))

for i in tqdm(range(num_portfolios)):
    weights = np.random.rand(len(assets))

    weights = weights - 0.5
    weights /= np.sum(np.abs(weights))


    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(annualized_cov_matrix, weights)))


    results[0, i] = portfolio_return
    results[1, i] = portfolio_std_dev
    results[2, i] = (portfolio_return - risk_free_rate_annualized) / portfolio_std_dev


    for j in range(len(weights)):
        results[j+3, i] = weights[j]


columns = ['Return', 'Volatility', 'Sharpe'] + list(assets)
results_df = pd.DataFrame(results.T, columns=columns)

best_sharpe_idx = results_df['Sharpe'].idxmax()
best_sharpe = results_df.iloc[best_sharpe_idx]['Sharpe']
best_return = results_df.iloc[best_sharpe_idx]['Return']
best_volatility = results_df.iloc[best_sharpe_idx]['Volatility']


negative_weights_mask = (results_df[assets] < 0).any(axis=1)

positive_weights_mask = ~negative_weights_mask & (results_df[assets] >= 0).all(axis=1)

best_positive_weights_idx = results_df[positive_weights_mask]['Sharpe'].idxmax()
best_positive_weights_sharpe = results_df.iloc[best_positive_weights_idx]['Sharpe']
best_positive_weights_return = results_df.iloc[best_positive_weights_idx]['Return']
best_positive_weights_volatility = results_df.iloc[best_positive_weights_idx]['Volatility']

plt.figure(figsize=(19, 6))

plt.scatter(results_df['Volatility'][negative_weights_mask],
            results_df['Return'][negative_weights_mask],
            c='grey', alpha=0.1, label="Negative Weight Portfolios", marker='o', s=1.5)

scatter = plt.scatter(results_df['Volatility'][~negative_weights_mask],
                      results_df['Return'][~negative_weights_mask],
                      c=results_df['Sharpe'][~negative_weights_mask], cmap='inferno', alpha=0.9, s=1.5)
plt.colorbar(scatter, label='Sharpe Ratio')

plt.scatter(best_volatility, best_return, color='black', marker='*', s=250, label="Best Monte Carlo Portfolio", edgecolors='black')
plt.scatter(best_positive_weights_volatility, best_positive_weights_return, color='blue', marker='*', s=250, label="Best Positive Weight Portfolio", edgecolors='blue')

plt.xlabel('Standard Deviation (Volatility)')
plt.ylabel('Annualized Return')
plt.title('Portfolio Return vs. Portfolio Volatility')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def plot_bar_chart(ax, weights, portfolio_name, sharpe_ratio):
    ax.bar(assets, weights, color=plt.cm.Paired.colors[:len(weights)])
    ax.set_xlabel('Assets')
    ax.set_ylabel('Weight')
    ax.set_title(f'{portfolio_name} Weights (Sharpe: {sharpe_ratio:.5f})')


    for i, weight in enumerate(weights):
        ax.text(i, weight + 0.02, f'{weight*100:.2f}%', ha='center', va='bottom', fontsize=10)

    ax.grid(True)


fig, axes = plt.subplots(1, 2, figsize=(18, 8))

best_monte_carlo_weights = results_df.iloc[best_sharpe_idx][assets].values
plot_bar_chart(axes[0], best_monte_carlo_weights, "Best Monte Carlo Portfolio", best_sharpe)

best_positive_weights = results_df.iloc[best_positive_weights_idx][assets].values
plot_bar_chart(axes[1], best_positive_weights, "Best Positive Weight Portfolio", best_positive_weights_sharpe)

plt.tight_layout()
plt.show()


# Are some Portfolios better than others after weights are optimized?

In [ ]:
def best_sharpe(assets):
  start_date = '2023-02-01'
  end_date = '2024-02-01'

  data = pd.DataFrame()

  for asset in assets:
    asset_data = yf.download(asset, start=start_date, end=end_date)['Close']
    data[asset] = asset_data

  returns = data.pct_change().dropna()
  mean_returns = returns.mean()
  cov_matrix = returns.cov()
  diag_sqrt = np.sqrt(np.diag(cov_matrix))
  normalized_cov_matrix = cov_matrix / np.outer(diag_sqrt, diag_sqrt)

  risk_free_rate_annualized = risk_free_rate
  annualized_cov_matrix = cov_matrix * 252

  num_portfolios = 3000 * len(assets)
  results = np.zeros((3 + len(assets), num_portfolios))

  for i in tqdm(range(num_portfolios)):

      weights = np.random.uniform(0, 1, size=len(assets))
      weights /= np.sum(weights)

      portfolio_return = np.sum(mean_returns * weights) * 252
      portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(annualized_cov_matrix, weights)))

      results[0, i] = portfolio_return
      results[1, i] = portfolio_std_dev
      results[2, i] = (portfolio_return - risk_free_rate_annualized) / portfolio_std_dev

      for j in range(len(weights)):
          results[j+3, i] = weights[j]

  columns = ['Return', 'Volatility', 'Sharpe'] + list(assets)
  results_df = pd.DataFrame(results.T, columns=columns)

  best_sharpe_idx = results_df['Sharpe'].idxmax()
  best_sharpe = results_df.iloc[best_sharpe_idx]['Sharpe']

  best_portfolio_weights = results_df.iloc[best_sharpe_idx][3:].values
  print("\n\nSharpe Ratio for best portfolio: " + str(best_sharpe))



portfolio = ["O", "JNJ"]
best_sharpe(portfolio)

### Some functions that will be used for the next section...

In [38]:
def portfolio_analysis(assets):
  start_date = '2024-02-01'
  end_date = '2025-02-01'

  data = pd.DataFrame()

  for asset in assets:
    asset_data = yf.download(asset, start=start_date, end=end_date)['Close']
    data[asset] = asset_data

  fig, axs = plt.subplots(1, len(assets), figsize=(19, 4))

  for i, asset in enumerate(assets):
    axs[i].plot(data.index, data[asset], label=asset)
    axs[i].set_title(f"{asset} Close Prices")
    axs[i].set_xlabel("Date")
    axs[i].set_ylabel("Close Price")
    axs[i].legend()
    axs[i].grid(True)

  plt.tight_layout()
  plt.show()

  returns = data.pct_change().dropna()
  mean_returns = returns.mean()
  cov_matrix = returns.cov()
  diag_sqrt = np.sqrt(np.diag(cov_matrix))
  normalized_cov_matrix = cov_matrix / np.outer(diag_sqrt, diag_sqrt)

  fig, axs = plt.subplots(1, 2, figsize=(21, 6))
  sns.barplot(x=mean_returns.index, y=mean_returns.values, ax=axs[0])
  axs[0].set_title('Mean Returns')
  axs[0].set_xlabel('Asset')
  axs[0].set_ylabel('Mean Return')
  axs[0].tick_params(axis='x', rotation=45)
  axs[0].grid(True)
  sns.heatmap(normalized_cov_matrix, annot=True, cmap='coolwarm', fmt="4f",vmin=-1, vmax=1, ax=axs[1])
  axs[1].set_title('Normalized Covariance Matrix - Returns')
  axs[1].set_xlabel('Asset')
  axs[1].set_ylabel('Asset')

def optimal_sharpe_ratio_portfolio(assets):
  start_date = '2024-02-01'
  end_date = '2025-02-01'

  data = pd.DataFrame()

  for asset in assets:
    asset_data = yf.download(asset, start=start_date, end=end_date)['Close']
    data[asset] = asset_data

  returns = data.pct_change().dropna()
  mean_returns = returns.mean()
  cov_matrix = returns.cov()
  diag_sqrt = np.sqrt(np.diag(cov_matrix))
  normalized_cov_matrix = cov_matrix / np.outer(diag_sqrt, diag_sqrt)

  risk_free_rate_annualized = risk_free_rate
  annualized_cov_matrix = cov_matrix * 252

  num_portfolios = 5000 * len(assets)
  results = np.zeros((3 + len(assets), num_portfolios))

  for i in tqdm(range(num_portfolios)):

      weights = np.random.uniform(0, 1, size=len(assets))
      weights /= np.sum(weights)

      portfolio_return = np.sum(mean_returns * weights) * 252
      portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(annualized_cov_matrix, weights)))

      results[0, i] = portfolio_return
      results[1, i] = portfolio_std_dev
      results[2, i] = (portfolio_return - risk_free_rate_annualized) / portfolio_std_dev

      for j in range(len(weights)):
          results[j+3, i] = weights[j]

  columns = ['Return', 'Volatility', 'Sharpe'] + list(assets)
  results_df = pd.DataFrame(results.T, columns=columns)

  best_sharpe_idx = results_df['Sharpe'].idxmax()
  best_sharpe = results_df.iloc[best_sharpe_idx]['Sharpe']
  best_return = results_df.iloc[best_sharpe_idx]['Return']
  best_volatility = results_df.iloc[best_sharpe_idx]['Volatility']

  best_portfolio_weights = results_df.iloc[best_sharpe_idx][3:].values
  print("\n\nSharpe Ratio for best portfolio: " + str(best_sharpe) + "\n\n")

  fig, axes = plt.subplots(1, 2, figsize=(20, 8))

  scatter = axes[0].scatter(results_df['Volatility'], results_df['Return'], c=results_df['Sharpe'], cmap='inferno', alpha=0.6, s=2)
  axes[0].scatter(best_volatility, best_return, color='black', marker='*', s=250, label="Best Monte Carlo Portfolio", edgecolors='black')
  axes[0].set_xlabel('Standard Deviation (Volatility)')
  axes[0].set_ylabel('Annualized Return')
  axes[0].set_title('Portfolio Return vs. Portfolio Volatility')
  axes[0].legend()
  axes[0].grid(True)
  fig.colorbar(scatter, ax=axes[0], label='Sharpe Ratio')

  axes[1].bar(assets, best_portfolio_weights, color='royalblue', alpha=0.8)
  axes[1].set_ylabel('Weight')
  axes[1].set_title('Optimal Portfolio Weights (Max Sharpe Ratio) = ' + str(best_sharpe))
  axes[1].set_ylim(0, 1)
  for i, weight in enumerate(best_portfolio_weights):
    axes[1].text(i, weight + 0.02, f"{weight:.2%}", ha='center', fontsize=12)

  plt.tight_layout()
  plt.show()

  print("Functions for game compiled")

# Lets Play a Game!


---
### Idea:
* We know different portfolios have different optimal sharpe ratios
* Is there a subset of stocks that outperform every other subset of stocks?

---
---
# Rules:
* **Whoever can construct a portfolio with the highest Sharpe Ratio wins!**

* Optimal weights will be calculated automatically
* Only positive weights will be considered (no short selling)
* Mean returns and covariance matrix will be calculated as discussed
* *Time limit: 15 mins*

---
---
### What to think about
* Will more stocks give me a better sharpe ratio than fewer? If so is there a point of dinminishing returns?
* What kind of stocks should I look to choose? How should my mean returns and covariance matrix ideally look?


In [ ]:
# This cell gives you a dashboard to do any analysis on your portfolio

# PORTFOLIO:
#--------#
portfolio = ["AAPL", "GOOGL"]
#--------#

portfolio_analysis(portfolio)

In [ ]:
# This cell calculates your sharpe ratio and weights (Male sure to record good results!)
optimal_sharpe_ratio_portfolio(portfolio)

# Closing Remarks
* Any questions, comments or concerns?
* For any additional question, I will stay for an additional *15 - 20  mins*.


---


* **Follow QFC on Instagram and Linkedin**
* **Follow Alpha Labs on Linkedin**
* **Check out Alpha Labs posters in JSOM Finance Lab**

---
---